# Overview

Notebook ini digunakan untuk Data Inspection & Data Preprocessing untuk data [**Online Retail II UCI**](https://www.kaggle.com/datasets/mashlyn/online-retail-ii-uci)

# Libraries

In [1]:
import pandas as pd
import numpy as np

# Import Data

In [2]:
df = pd.read_csv("data/raw/online_retail_II.csv") # Load csv data
df.sample(5) # Preview data

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
64845,495184,84623,PINK GINGHAM ROSE FLOOR CUSHION,4,2010-01-21 13:58:00,7.95,17841.0,United Kingdom
926368,571408,23501,KEY RING BASEBALL BOOT UNION JACK,1,2011-10-17 12:22:00,1.25,15012.0,United Kingdom
760489,557621,23174,REGENCY SUGAR BOWL GREEN,4,2011-06-21 14:00:00,4.15,12444.0,Norway
555280,538827,82613B,"METAL SIGN,CUPCAKE SINGLE HOOK",20,2010-12-14 12:59:00,0.42,14298.0,United Kingdom
946320,572911,84974,S/2 ZINC HEART DESIGN PLANTERS,1,2011-10-26 16:03:00,9.95,16345.0,United Kingdom


Data Description (From Kaggle):
- **InvoiceNo** (**Nominal**): Invoice number. A 6-digit integer uniquely assigned to each transaction. If this code starts with the letter 'c', it indicates a cancellation.
- **StockCode** (**Nominal**): Product (item) code. A 5-digit integer uniquely assigned to each distinct product.
- **Description** (**Nominal**): Product (item) name.
- **Quantity** (**Numeric**): The quantities of each product (item) per transaction.
- **InvoiceDate** (**Date**): Invoice datetime. The day and time when a transaction was generated.
- **Price** (**Numeric**): Unit price. Product price per unit in sterling (£).
- **CustomerID** (**Nominal**): Customer number. A 5-digit integer uniquely assigned to each customer.
- **Country** (**Nominal**): Country name. The name of the country where a customer resides.

Setelah melihat data secara sekilas dan mencocokan dengan Data Description, muncul pertanyaan: kenapa Customer ID memiliki format float/decimal? dan apakah kolom kolom lainnya sudah mempunyai format yang sesuai?

# Data Inspection 

Mari kita coba cek lebih detail dan lihat apakah ada data format yang tidak sesuai atau keanehan lainnya di data

## Data Structure

In [3]:
df.info() # dtypes + non-null counts + data type

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1067371 entries, 0 to 1067370
Data columns (total 8 columns):
 #   Column       Non-Null Count    Dtype  
---  ------       --------------    -----  
 0   Invoice      1067371 non-null  object 
 1   StockCode    1067371 non-null  object 
 2   Description  1062989 non-null  object 
 3   Quantity     1067371 non-null  int64  
 4   InvoiceDate  1067371 non-null  object 
 5   Price        1067371 non-null  float64
 6   Customer ID  824364 non-null   float64
 7   Country      1067371 non-null  object 
dtypes: float64(2), int64(1), object(5)
memory usage: 65.1+ MB


- Terdapat 1.067.371 baris x 8 kolom data
- Ada null values pada `Description` dan `Customer ID`
- Data type untuk **Quantity**, **Description**, **Price**, **Country** sudah **sesuai**
- **Invoice** dan **StockCode** masih dianggap string, perlu inspeksi lebih lanjut apakah sudah sesuai
- **InvoiceDate** masih berbentuk string, perlu dikonversi
- **Customer ID** masih berbentuk float, seharusnya integer juga cukup
- Overall, penamaan kolom belum ter-*standardize*, kita dapat me-*rename* nya agar lebih intuitif dan mudah dibaca

## Rename Column

In [4]:
# Replace nama kolom sesuai dengan list, pastikan urutan sesuai
df.columns = ['order_id','sku_id','sku_name','quantity','order_date','sku_price','customer_id','country']

Sekarang, penamaan kolom lebih intuitif, lebih mudah dibaca, dan tidak rentan untuk typo pada saat menulis kode (tidak perlu ingat ingat StockCode c nya besar atau kecil)

## Correcting Data Format

Akan kita lakukan:
- Pengecekan terhadap data yang tidak sesuai deskripsi atau bukan dalam format seharusnya (contoh: sku_id seharusnya 5 digit, order_id 5 digit, order_id 6 digit, dsb)
- Mengubah format data agar sesuai deskripsi

### order_id
Salah satu hint yang diberikan di data description adalah, jika order_id start dengan huruf 'c', maka order tersebut di cancel, dan seharusnya order_id adalah integer 6 digit. Salah satu cara yang dapat digunakan adalah dengan mengecek apakah suatu order_id dapat diconvert ke integer apa tidak

In [5]:
# Fungsi untuk cek apakah value yang diberikan bisa diconvert ke integer
def is_convertible_to_int(value):
    try: # Try to convert to int
        int(value) # Jika tidak bisa, akan mengeluarkan error
        return True # Jika aman, maka return True
    except ValueError: # Jika Error
        return False # return False

In [6]:
mask = df['order_id'].apply(lambda x: is_convertible_to_int(x)) # Memberikan flag untuk setiap record apakah dapat di convert ke integer atau tidak
temp = df[~mask] # Cek record yang order_id nya tidak bisa di convert ke integer
temp

,order_id,sku_id,sku_name,quantity,order_date,sku_price,customer_id,country
178,C489449,22087,PAPER BUNTING WHITE LACE,-12,2009-12-01 10:33:00,2.95,16321.0,Australia
179,C489449,85206A,CREAM FELT EASTER EGG BASKET,-6,2009-12-01 10:33:00,1.65,16321.0,Australia
180,C489449,21895,POTTING SHED SOW 'N' GROW SET,-4,2009-12-01 10:33:00,4.25,16321.0,Australia
181,C489449,21896,POTTING SHED TWINE,-6,2009-12-01 10:33:00,2.10,16321.0,Australia
182,C489449,22083,PAPER CHAIN KIT RETRO SPOT,-12,2009-12-01 10:33:00,2.95,16321.0,Australia
...,...,...,...,...,...,...,...,...
1065910,C581490,23144,ZINC T-LIGHT HOLDER STARS SMALL,-11,2011-12-09 09:57:00,0.83,14397.0,United Kingdom
1067002,C581499,M,Manual,-1,2011-12-09 10:28:00,224.69,15498.0,United Kingdom
1067176,C581568,21258,VICTORIAN SEWING BOX LARGE,-5,2011-12-09 11:57:00,10.95,15311.0,United Kingdom
1067177,C581569,84978,HANGING HEART JAR T-LIGHT HOLDER,-1,2011-12-09 11:58:00,1.25,17315.0,United Kingdom


Dari sini kita bisa lihat bahwa:
- benar, ada order_id yang tercancel
- ada sku_id yang hanya tertuliskan 'M', dan sku_name nya 'Manual'. Berarti ada kemungkinan penulisan sku_id yang aneh lainnya (seharusnya 5 digit)
- Untuk quantity yang di cancel, besarannya negatif, ini bisa jadi indikasi item retur

Mari coba cek kembali apakah ada order_id dengan format aneh lainnya (selain cancel)

In [7]:
mask = temp['order_id'].apply(lambda x: x[0]!='C') # Cek apakah ada order_id lain yang tidak berawalan (posisi 0) C
temp[mask]

,order_id,sku_id,sku_name,quantity,order_date,sku_price,customer_id,country
179403,A506401,B,Adjust bad debt,1,2010-04-29 13:36:00,-53594.36,NaN,United Kingdom
276274,A516228,B,Adjust bad debt,1,2010-07-19 11:24:00,-44031.79,NaN,United Kingdom
403472,A528059,B,Adjust bad debt,1,2010-10-20 12:04:00,-38925.87,NaN,United Kingdom
825443,A563185,B,Adjust bad debt,1,2011-08-12 14:50:00,11062.06,NaN,United Kingdom
825444,A563186,B,Adjust bad debt,1,2011-08-12 14:51:00,-11062.06,NaN,United Kingdom
825445,A563187,B,Adjust bad debt,1,2011-08-12 14:52:00,-11062.06,NaN,United Kingdom


Ternyata ada order_id yang berawalan A, yang berisi transaksi untuk *adjust* hutang

Setelah menemukan 2 macam order_id yang tidak sesuai format (cancellation dan adjust bad debt), kita dapat mem-*flag* transaksi tersebut dan men-*decide* nanti untuk *handling* data tersebut (ex: kita dapat melakukan analisa untuk 'kenapa order tersebut di-*cancel*? produk apa yang rentan di-*cancel*?)

Untuk sekarang, mari flagging order_id tersebut

In [8]:
df["order_id"]   = df["order_id"].astype(str).str.strip() # Make sure untuk menghilangkan extra space (just in case)
df["order_id_cancelled"] = df["order_id"].str.upper().str.startswith("C") # order_id cancelled kalau depannya C
df["order_id_adjustment"] = df["order_id"].str.upper().str.startswith("A") # order_id adjusted kalau depannya A

Terakhir, mari cek apakah order_id yang cancelled tersebut merupakan suatu *follow-up* dari order_id lain (misal, C489449 merupakan cancellation dari order 489449)

In [9]:
cancelled_order_list = df[df['order_id_cancelled']]['order_id'] # Ambil order_id yang tercancel
cancelled_order_list = [x[1:] for x in cancelled_order_list] # Ambil karakter setelah huruf pertama (C)
df[df['order_id'].isin(cancelled_order_list)] # Cek apakah ada order_id yang memiliki id tersebut

,order_id,sku_id,sku_name,quantity,order_date,sku_price,customer_id,country,order_id_cancelled,order_id_adjustment


Ternyata tidak ada, jadi kita tidak tahu cancellation tersebut dari order yang mana

### sku_id

Let's follow the same flow as before, but this time, let's take the unique combination of sku_id and sku_name

In [10]:
catalog = df[['sku_id','sku_name']].drop_duplicates() # drop duplikat record by sku_id dan sku_name
catalog

,sku_id,sku_name
0,85048,15CM CHRISTMAS GLASS BALL 20 LIGHTS
1,79323P,PINK CHERRY LIGHTS
2,79323W,WHITE CHERRY LIGHTS
3,22041,"RECORD FRAME 7"" SINGLE SIZE"
4,21232,STRAWBERRY CERAMIC TRINKET BOX
...,...,...
1062371,23090,missing
1063082,85123A,CREAM HANGING HEART T-LIGHT HOLDER
1064015,85175,NaN
1064386,23169,smashed


Kita bisa lihat bahwa ada beberapa penamaan yang aneh:
- sku_id, ada huruf dibelakang format 5 digit. Apakah ini hanya sebuah varian dari SKU yang sama?
- sku_name yang hanya missing atau smashed

Akan dilakukan flow yang serupa dengan order_id

In [11]:
mask = catalog['sku_id'].apply(lambda x: is_convertible_to_int(x)) # Memberikan flag sku_id yang dapat di convert ke int dan tidak dapat di convert ke int
temp = catalog[~mask] # Filter catalog dengan sku_id yang tidak dapat di convert ke int
temp

,sku_id,sku_name
1,79323P,PINK CHERRY LIGHTS
2,79323W,WHITE CHERRY LIGHTS
12,48173C,DOOR MAT BLACK FLOCK
23,35004B,SET OF 3 BLACK FLYING DUCKS
28,84596F,SMALL MARSHMALLOWS PINK BOWL
...,...,...
1049836,84876B,found
1060595,84801A,check
1060784,84508C,check
1060792,72801C,check


Mari kita lihat apakah masing masing sku_id dengan huruf dibelakangnya memiliki *counter-part*

In [12]:
temp.columns = ['other_sku_id','other_sku_name']
temp['sku_id'] = temp['other_sku_id'].apply(lambda x: ''.join(filter(str.isdigit, x)))
catalog.merge(temp,on=['sku_id'])

C:\Users\PC\AppData\Local\Temp\ipykernel_8968\1397452419.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  temp['sku_id'] = temp['other_sku_id'].apply(lambda x: ''.join(filter(str.isdigit, x)))


,sku_id,sku_name,other_sku_id,other_sku_name
0,10002,INFLATABLE POLITICAL GLOBE,10002R,ROBOT PENCIL SHARPNER
1,47566,PARTY BUNTING,47566B,TEA TIME PARTY BUNTING
2,47566,PARTY BUNTING,47566B,missing
3,47566,PARTY BUNTING,47566b,TEA TIME PARTY BUNTING
4,47566,PARTY BUNTING,47566B,NaN
5,47566,PARTY BUNTING,47566B,correct previous adjustment
6,47566,PARTY BUNTING,47566B,stock credited from royal yacht inc
7,47566,PARTY BUNTING,47566B,incorrectly credited C550456 see 47
8,47566,PARTY BUNTING,47566B,reverse previous adjustment
9,84292,NaN,84292B,BLUE COL METAL FOLDING CHAIR


Penamaan SKU nya sangat berantakan, kapital huruf tidak konsisten, dan sedikit relasi antara kode original dengan yang ada huruf dibelakangnya

Satu sku_id pun bisa punya beberapa nama, mari kita cek di data order

In [13]:
# Fix untuk extra space, double space, make upper case
df['sku_id'] = df['sku_id'].astype(str).str.upper().str.strip() # Upper case & remove extra space
df['sku_id'] = df['sku_id'].apply(lambda x: x.replace('  ',' ')) # remove double space
df['sku_name'] = df['sku_name'].astype(str).str.upper().str.strip()
df['sku_name'] = df['sku_name'].apply(lambda x: x.replace('  ',' '))

Lalu, mari kita revisi catalog

In [14]:
# Count jumlah order per sku_id x sku_name
catalog = df.groupby(['sku_id','sku_name']).agg({'order_id':'count'}).reset_index().sort_values(['sku_id','order_id'],ascending=[True,False])

# Cek sku_id mana yang punya lebih dari 1 nama
catalog['sku_id'].value_counts()

sku_id
20713    9
23084    8
22423    8
22734    8
21830    7
        ..
22041    1
22040    1
22039    1
22038    1
23274    1
Name: count, Length: 5131, dtype: int64

Ada nama sku yang aneh lagi, GIFT, tidak memiliki digit sama sekali

In [15]:
# Cek salah satu sku yang memiliki nama > 1
catalog[catalog['sku_id']=='20713']

,sku_id,sku_name,order_id
509,20713,JUMBO BAG OWLS,1372
512,20713,NAN,5
508,20713,FOUND,2
510,20713,MARKED AS 23343,1
511,20713,MISSING,1
513,20713,WRONGLY CODED 23343,1
514,20713,WRONGLY CODED-23343,1
515,20713,WRONGLY MARKED 23343,1
516,20713,WRONGLY MARKED. 23343 IN BOX,1


Setelah pengecekan, sepertinya kasus yang penamaannya aneh ini (semoga) cukup sedikit
Kita dapat fokus ke sku name dengan frekuensi tertinggi (modus) sebagai sku name yang *valid*

In [16]:
valid_catalog = catalog.sort_values('order_id',ascending=False).groupby('sku_id').head(1) # Sort berdasarkan order_id terbanyak, dan ambil 1 teratas per sku_id
valid_catalog

,sku_id,sku_name,order_id
7855,85123A,WHITE HANGING HEART T-LIGHT HOLDER,5918
3114,22423,REGENCY CAKESTAND 3 TIER,4412
7807,85099B,JUMBO BAG RED RETROSPOT,3469
7303,84879,ASSORTED COLOUR BIRD ORNAMENT,2958
5649,47566,PARTY BUNTING,2765
...,...,...,...
5188,35948,NAN,1
5189,35949,NAN,1
5190,35950,FOLKART HEART CHRISTMAS DECORATIONS,1
5201,35956,LOST,1


Mari cek sku_id yang tidak ada digit

In [17]:
valid_catalog['sku_id_no_digit'] = valid_catalog['sku_id'].apply(lambda x: not any(char.isdigit() for char in x)) # Memberikan flag untuk sku_id yang tidak ada digit nya
valid_catalog[valid_catalog['sku_id_no_digit']] # cek sku_id yang tidak punya digit

,sku_id,sku_name,order_id,sku_id_no_digit
8773,POST,POSTAGE,2115,True
8750,DOT,DOTCOM POSTAGE,1444,True
8770,M,MANUAL,1426,True
8701,D,DISCOUNT,177,True
8774,S,SAMPLES,104,True
8696,BANK CHARGES,BANK CHARGES,102,True
8694,AMAZONFEE,AMAZON FEE,43,True
8690,ADJUST,ADJUSTMENT BY JOHN ON 26/01/2010 16,38,True
8747,DCGSSGIRL,GIRLS PARTY BAG,23,True
8744,DCGSSBOY,BOYS PARTY BAG,21,True


Sepertinya sku_id yang tidak punya digit adalah extra fee, kita dapat melakukan flagging jika ingin menganalisa extra fee tersebut (seperti berapa pengeluaran buat postage, amazon fee, bank charges, etc)

Sekarang, join kembali dengan data order, jadi kita hanya akan memperhitungkan data order yang valid sku

In [18]:
# Join dengan data order
df = df.merge(valid_catalog.drop(columns={'order_id'}),on=['sku_id','sku_name'])

### customer_id

Untuk customer_id, concern utama ada di format yang seharusnya integer dan juga banyaknya data yang null

In [19]:
# Convert to Int64 in case ada null values
df['customer_id'] = df['customer_id'].astype("Int64")

### order_date
Untuk order date, problemnya ada di format yang belum datetime

In [20]:
# Convert ke datetime
df["order_date"] = pd.to_datetime(df["order_date"])

### country
country hanya perlu dipastikan bahwa valuenya seragam (upper) untuk menghindari perbedaan penulisan (ex: indonesia vs Indonesia)
dan juga pengecekan unique values untuk mengecek apakah yang dituliskan merupakan valid countries

In [21]:
# Convert str, hilangkan white space, convert ke uppercase
df['country'] = df['country'].astype(str).str.strip().str.upper()
df['country'].unique()

array(['UNITED KINGDOM', 'FRANCE', 'USA', 'BELGIUM', 'AUSTRALIA', 'EIRE',
       'GERMANY', 'PORTUGAL', 'JAPAN', 'DENMARK', 'NIGERIA',
       'NETHERLANDS', 'POLAND', 'SPAIN', 'CHANNEL ISLANDS', 'ITALY',
       'CYPRUS', 'GREECE', 'NORWAY', 'AUSTRIA', 'SWEDEN',
       'UNITED ARAB EMIRATES', 'FINLAND', 'SWITZERLAND', 'UNSPECIFIED',
       'MALTA', 'BAHRAIN', 'RSA', 'BERMUDA', 'HONG KONG', 'SINGAPORE',
       'THAILAND', 'ISRAEL', 'LITHUANIA', 'WEST INDIES', 'LEBANON',
       'KOREA', 'BRAZIL', 'CANADA', 'ICELAND', 'SAUDI ARABIA',
       'CZECH REPUBLIC', 'EUROPEAN COMMUNITY'], dtype=object)

untuk country, tidak ada format yang aneh, hanya data null seperti UNSPECIFIED

### quantity
Kita perlu cek untuk quantity yang negatif

In [22]:
# Filter record yang memiliki qty dibawah 0
df[df['quantity']<0]

,order_id,sku_id,sku_name,quantity,order_date,sku_price,customer_id,country,order_id_cancelled,order_id_adjustment,sku_id_no_digit
145,C489449,22087,PAPER BUNTING WHITE LACE,-12,2009-12-01 10:33:00,2.95,16321,AUSTRALIA,True,False,False
146,C489449,85206A,CREAM FELT EASTER EGG BASKET,-6,2009-12-01 10:33:00,1.65,16321,AUSTRALIA,True,False,False
147,C489449,21895,POTTING SHED SOW 'N' GROW SET,-4,2009-12-01 10:33:00,4.25,16321,AUSTRALIA,True,False,False
148,C489449,21896,POTTING SHED TWINE,-6,2009-12-01 10:33:00,2.10,16321,AUSTRALIA,True,False,False
149,C489449,21871,SAVE THE PLANET MUG,-12,2009-12-01 10:33:00,1.25,16321,AUSTRALIA,True,False,False
...,...,...,...,...,...,...,...,...,...,...,...
1001066,C581490,23144,ZINC T-LIGHT HOLDER STARS SMALL,-11,2011-12-09 09:57:00,0.83,14397,UNITED KINGDOM,True,False,False
1002127,C581499,M,MANUAL,-1,2011-12-09 10:28:00,224.69,15498,UNITED KINGDOM,True,False,True
1002295,C581568,21258,VICTORIAN SEWING BOX LARGE,-5,2011-12-09 11:57:00,10.95,15311,UNITED KINGDOM,True,False,False
1002296,C581569,84978,HANGING HEART JAR T-LIGHT HOLDER,-1,2011-12-09 11:58:00,1.25,17315,UNITED KINGDOM,True,False,False


Sekilas, sepertinya quantity negatif ada pada order_id_cancelled, mari kita cek yang selain itu

In [23]:
# Filter dataframe yang qty < 0 dan bukan karena di cancel
df[(df['quantity']<0)&(~df['order_id_cancelled'])]

,order_id,sku_id,sku_name,quantity,order_date,sku_price,customer_id,country,order_id_cancelled,order_id_adjustment,sku_id_no_digit
399,489521,21646,NAN,-50,2009-12-01 11:44:00,0.0,<NA>,UNITED KINGDOM,False,False,False
2663,489655,20683,NAN,-44,2009-12-01 17:26:00,0.0,<NA>,UNITED KINGDOM,False,False,False
2704,489660,35956,LOST,-1043,2009-12-01 17:43:00,0.0,<NA>,UNITED KINGDOM,False,False,False
2710,489663,35605A,DAMAGES,-117,2009-12-01 18:02:00,0.0,<NA>,UNITED KINGDOM,False,False,False
13844,490736,35983,NAN,-200,2009-12-07 17:13:00,0.0,<NA>,UNITED KINGDOM,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...
746721,561249,DCGS0073,EBAY,-4,2011-07-26 11:51:00,0.0,<NA>,UNITED KINGDOM,False,False,False
746722,561250,DCGS0071,NAN,-2,2011-07-26 11:51:00,0.0,<NA>,UNITED KINGDOM,False,False,False
746723,561255,DCGS0066P,NAN,-3,2011-07-26 11:52:00,0.0,<NA>,UNITED KINGDOM,False,False,False
746724,561254,DCGS0067,EBAY,-11,2011-07-26 11:52:00,0.0,<NA>,UNITED KINGDOM,False,False,False


Sepertinya sku_price yang memiliki quantity negatif bernilai 0, mari kita pastikan

In [24]:
# Jumlah sku_price dari order yang tidak tercancel, dan punya quantity negatif
df[(df['quantity']<0)&(~df['order_id_cancelled'])]['sku_price'].sum()

0.0

Kita bisa takeout data tersebut

In [25]:
mask = ~ ((df['quantity']<0)&(~df['order_id_cancelled'])) # Negasi dari kondisi diatas
df = df[mask]

# Save the cleaned data

In [26]:
# Save it as parquet to preserve the formatting and have smaller data size
df.to_parquet('data/preprocessed/cleaned_data.parquet') 